Auto Complete using N-Grams
<br><br>
**🛑The following is still in work and not completed !!**

In [1]:
#Initial imports done

import math
import nltk
import random
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt



In [2]:
# Since the data is in text format
data_dir = "/content"
file_path = data_dir + "/en_US.twitter.txt"

## nltk settings
nltk.data.path.append(data_dir)
nltk.download('punkt')

with open(file_path,'r') as filedata:
  textdata = filedata.read()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
f1 = open(file_path,'r')
count =0
for i in f1.readlines():
  if(count <10):
    print(i)
    print("\n")

  count+=1


How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.



When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.



they've decided its more fun if I don't.



So Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)



Words from a complete stranger! Made my birthday even better :)



First Cubs game ever! Wrigley field is gorgeous. This is perfect. Go Cubs Go!



i no! i get another day off from skool due to the wonderful snow (: and THIS wakes me up...damn thing



I'm coo... Jus at work hella tired r u ever in cali



The new sundrop commercial ...hehe love at first sight



we need to reconnect THIS WEEK





### Preprocessing Pipeline
1. Split the data by \n charachter
2. Remove leading and trailing spaces
3. Remove Empty sentences
4. Tokenise sentences using nltk tokenize

In [4]:
#Returns list
def preprocess_pipeline(textdata)-> 'list':
  sentence = textdata.split('\n')

  sentence = [s.strip() for s in sentence]

  sentence = [s for s in sentence if len(s) >0]

  #Tokenize

  tokenize=[]

  for i in sentence:
    i = i.lower()
    token = nltk.word_tokenize(i)
    tokenize.append(token)

  #print(f'Tokenize format :{tokenize[0]}')
  return tokenize

tokenized_sentence = preprocess_pipeline(textdata)

### Split into Train,Test datasets

In [5]:
Train,Test = train_test_split(tokenized_sentence,test_size=0.2,random_state = 42)

Train, validation = train_test_split(Train, test_size=0.25, random_state=42)

## 🧹Cleaning the data
1. Creating the frequency dictonery
2. Creating a Closed Vocabulary - We need to deal with Out-of-vocabulary words. This helps the model to handle words which are not present in the training corpus. 
3. Adding UNK tokens - In this function we'll add <unk> tokens, to those words which are not in the closed_vocabulary which we just made
4. Final Cleaning Pipeline

In [6]:
# Frequency dictionery :

def frequency_dictionery(sentences)->'dict':
  word_count = {}

  for sent in sentences:
    for token in sent:

      if (token not in word_count.keys()):
        word_count[token] = 1

      else:
        word_count[token]+=1

  return word_count


In [7]:
'''
 This function creates a closed vocabulary containing only those words 
 according to the count_threshold parameter.
'''

def closed_vocabulary(tokenized_sentence,count_threshold)-> 'list':

  cv =[]
  #Get the frequency dictionery from abpove function
  word_count = frequency_dictionery(tokenized_sentence)

  for word,count in word_count.items():
    if (count >= count_threshold):
      cv.append(word)

  return cv


In [8]:
# Adding UNK tokens
# This function returns a list
def unk_tokenize(tokenized_sentences, vocabulary, unknown_token = "<unk>")-> 'list':

  # Convert Vocabulary into a set
  vocabulary = set(vocabulary)

  # Create empty list for sentences
  new_tokenized_sentences = []
  
  # Iterate over sentences
  for sentence in tokenized_sentences:

    # Iterate over sentence and add <unk> 
    # if the token is absent from the vocabulary
    new_sentence = []
    for token in sentence:
      if token in vocabulary:
        new_sentence.append(token)
      else:
        new_sentence.append(unknown_token)
    
    # Append sentece to the new list
    new_tokenized_sentences.append(new_sentence)

  return new_tokenized_sentences



In [9]:
def final_cleansing_pipeline(train,test,count_threshold):

  #Closed vocabulary
  vocabulary = closed_vocabulary(train,count_threshold)

  new_train = unk_tokenize(train,vocabulary)
  new_test = unk_tokenize(test,vocabulary)

  return new_test,new_train,vocabulary
  


In [10]:
frequency = 6
final_test,final_train,vocabulary = final_cleansing_pipeline(Train,Test,frequency)

##  🔨 Building the model

In [11]:
# This function returns a mapping from n-grams to their frequency in the dataset.

#Return dictionery
def count_n_grams(data,n,start_token = "<s>", end_token = "<e>")-> 'dict':
  ngrams ={}
  for sentence in data:
    # Append n start tokens and a single end token to the sentence
    sentence = [start_token]*n + sentence + [end_token]
    sentence = tuple(sentence)

    length1 = len(sentence) if n==1 else len(sentence)-1

    for i in range(length1):
      # Get the n-grams
      n_gram = sentence[i:i+n]

      if n_gram in ngrams.keys():
        ngrams[n_gram] +=1

      else:
        ngrams[n_gram]=1

  return ngrams

This function calculates the priority for the next word given the prior n-gram.

## K Smoothning
if we come across a n-gram that wasn't in the training set, then we have our denominator as 0 which raises an error.<br>
<br>
Thus, we use k-smoothing, which adds a positive constant  k  to each numerator and  k×|V|  in the denominator, where  |V|  is the number of words in the vocabulary. This ensures any n-gram with zero count has the same probability of  1|V| 

In [12]:
# Returns probablity of float
def getProbablity_single_word(word,previous_ngram,ngCount,nplus1_gram_counts, vocabulary_size, k = 1.0) -> 'float':

  previous_ngram = tuple(previous_ngram)

  previous_ngram_count = ngCount[previous_ngram] if previous_ngram in ngCount else 0

  #The denominator - We need to take care
  denom = previous_ngram_count + k * vocabulary_size

  nplus1_gram = previous_ngram + (word,)

  # Calculating the nplus1_gram count .If it exist in frequency dict or it is 0 
  nplus1_count =nplus1_gram_counts[nplus1_gram] if nplus1_gram in nplus1_gram_counts else 0

  numerator = nplus1_count + k   #Numerator

  #Final fraction
  final_prob = (numerator/denom)
  return final_prob


Loop over all the words in the vocabulary and then compute their probabilites using our getProbablity_single_word() fn.

In [13]:
#Returns Dictionery
def prob(previous_ngram,ngCount,np1Count,vocab,k=1.0) -> 'dict':
  previous_ngram = tuple(previous_ngram)
  vocab = vocab + ["<e>", "<unk>"]
  vsize = len(vocab)
  probablities ={}

  for words in vocab:
    probab = getProbablity_single_word(words,previous_ngram,ngCount,np1Count,vsize,k=k)

    #Mapping: word->probablity
    probablities[words] = probab
    
  return probablities

## The Main System

In [14]:
def auto_complete(previous_token,ngCount,np1_count,vocab,k=1.0,start_with=None):
  n = len(list(ngCount.keys())[0])
  previous_ngram = previous_token[-n:]

  #Calculate the probablities of all word
  probablities = prob(previous_ngram,ngCount,np1_count,vocab,k=k)

  suggest =None
  maxProb =0

  #Get the words with maximum probablity
  for word,prob1 in probablities.items():
    if start_with != None:
      if not word.startswith(start_with):
        continue

    if prob1>maxProb:
      suggest = word
      maxProb = prob1

  return suggest,maxProb


## Getting multiple suggestions

In [15]:
def multiple_suggestion(previous_token,ngram_clist,vocab,k=1.0,start_with=None):

  cnt = len(ngram_clist)
  suggest =[]

  for i in range(cnt-1):
    ngcount = ngram_clist[i]
    np1_gram_count = ngram_clist[i+1]

    suggestion = auto_complete(previous_token,ngcount,np1_gram_count,vocab,k=k,start_with=start_with)
    suggest.append(suggestion)

  return suggest

## ✔️ Almost done!!

In [16]:
ngram_clist = []
for i in range(1,6):
  n_models = count_n_grams(final_train,i)
  ngram_clist.append(n_models)
  

In [20]:
previous_tokens = ["i", "wonder", "why"]
suggestions = multiple_suggestion(previous_tokens,ngram_clist,vocabulary,k=1.0)
display(suggestions)

[('do', 0.012574575493345572),
 ('i', 0.0007262917617763021),
 ('.', 0.0002079002079002079),
 ('check', 0.00010400416016640666)]